In [0]:
from bs4 import BeautifulSoup
from datetime import datetime

import pandas as pd
import requests
import re

In [0]:
class get_info():
    def __init__(self, query, s_date, e_date, maxpage):
        self.query  = query
        self.s_date = s_date
        self.e_date = e_date
        self.maxpage = maxpage
    
    # 특정 키워드의 기사 검색 (DataFrame 저장)
    def crawler(self):
        
        s_from = self.s_date.replace(".","")
        e_to   = self.e_date.replace(".","")
        page = 1
        maxpage_t = (int(self.maxpage)-1)*10+1          # 11= 2페이지 21=3페이지 31=4페이지 

        # 뉴스 및 url 저장을 위한 경로 
        df = pd.DataFrame(columns=["date", "press", "title", "contents", "link"])
        urls = []
        
        cnt = 0
        while page < maxpage_t:

            url = "https://search.naver.com/search.naver?where=news&query=" + query + "&photo=3&field=0&reporter_article=&pd=3"+"&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
            req = requests.get(url)
        
            cont = req.content
            soup = BeautifulSoup(cont, 'html.parser')

            # 처음에만, 기사 건수, 몇 페이지인지 확인    
            if page == 1:
                num = soup.select("#main_pack > div > div.section_head > div.title_desc.all_my > span")
                num2 = num[0].text.split("/")
                prs_num = int((num2[0].split("-"))[-1])
                tot_num = int((num2[-1].split("건"))[0].replace(",",""))

                totpage = tot_num // prs_num if (tot_num % prs_num) == 0 else tot_num // prs_num +1
                maxiter = (totpage-1)*10+1    
                
            print("page/total page = %d/ %d" %(page//10+1, totpage))
            
            # 기사 날짜, 언론사, 제목, 내용, 요약 크롤링
            for urls in soup.select("._sp_each_url"):
                ## 네이버 뉴스만(크롤링할 수 있는 형식)
                if urls["href"].startswith("https://news.naver.com"):
                    
                    news_detail   = self.get_news(urls["href"])
                    if news_detail == 'pass' : continue
                    
                    # 크롤링한 내용 저장
                    df = df.append(pd.DataFrame([[news_detail[0], news_detail[1], news_detail[2], news_detail[3], urls["href"]]],
                                                  columns=["date","press","title","contents","link"]),ignore_index=True)
                    cnt += 1
            
            print('  ', cnt, '건의 뉴스가 저장 됨')        
            if page == maxiter : break
            page += 10

        return df

    def get_news(self, n_url):
        news_detail = []
        breq = requests.get(n_url)
        bsoup = BeautifulSoup(breq.content, 'html.parser')

        #기사 날짜
        try: 
            pdate = bsoup.select('.t11')[0].get_text()[:11]
            news_detail.append(pdate)
        except:
            return 'pass'

        #언론사
        press = bsoup.select('div.press_logo > a > img')[0].get('title')
        news_detail.append(press)
        
        #기사 제목
        title = bsoup.select('h3#articleTitle')[0].text #대괄호는 h3#articleTitle 인 것중 첫번째 그룹만 가져오겠다.
        news_detail.append(title)

        #기사 내용
        _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
        btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
        news_detail.append(btext.strip())
        
        #기사 url
        news_detail.append(n_url)

        return news_detail   

In [0]:
query  = "코로나"
s_date = "2020.04.01"
e_date = "2020.04.10"
maxpage = 101

news1 = get_info(query, s_date, e_date, maxpage)
df1 = news1.crawler()

page/total page = 1/ 959
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
   17 건의 뉴스가 저장 됨
page/total page = 2/ 959
2020.04.10.
2020.04.10.
2020.04.09.
2020.04.09.
2020.04.10.
2020.04.09.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
   28 건의 뉴스가 저장 됨
page/total page = 3/ 959
2020.04.09.
2020.04.10.
2020.04.09.
2020.04.10.
2020.04.10.
2020.04.09.
2020.04.10.
2020.04.09.
2020.04.10.
2020.04.09.
2020.04.09.
   39 건의 뉴스가 저장 됨
page/total page = 4/ 959
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.08.
2020.04.08.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
2020.04.10.
   51 건의 뉴스가 저장 됨
page/total page = 5/ 959
2020.04.10.
2020.04.08.
2020.04.09.
2020.04.09.
2020.04.09.
2020.04.09.
2020.04.09.
2020.04.09.
2020.04.08.
2020.04.08.
2020.04.10.
2020.04.10.
2020.04.09.
2020.04.10.
   65 건의 뉴스가 저장 됨
page/

page/total page = 49/ 959
2020.04.06.
2020.04.09.
2020.04.09.
2020.04.08.
2020.04.06.
2020.04.06.
2020.04.08.
2020.04.10.
2020.04.08.
2020.04.07.
   515 건의 뉴스가 저장 됨
page/total page = 50/ 959
2020.04.09.
2020.04.10.
2020.04.09.
2020.04.10.
2020.04.06.
2020.04.07.
2020.04.06.
2020.04.06.
2020.04.08.
2020.04.08.
   525 건의 뉴스가 저장 됨
page/total page = 51/ 959
2020.04.09.
2020.04.08.
2020.04.09.
2020.04.10.
2020.04.09.
2020.04.09.
2020.04.06.
2020.04.06.
2020.04.09.
2020.04.07.
   535 건의 뉴스가 저장 됨
page/total page = 52/ 959
2020.04.08.
2020.04.07.
2020.04.08.
2020.04.09.
2020.04.07.
2020.04.07.
2020.04.05.
2020.04.08.
2020.04.06.
2020.04.07.
   545 건의 뉴스가 저장 됨
page/total page = 53/ 959
2020.04.09.
2020.04.08.
2020.04.06.
2020.04.09.
2020.04.07.
2020.04.08.
2020.04.09.
2020.04.08.
2020.04.08.
   554 건의 뉴스가 저장 됨
page/total page = 54/ 959
2020.04.08.
2020.04.10.
2020.04.08.
2020.04.09.
2020.04.07.
2020.04.07.
2020.04.08.
2020.04.08.
2020.04.09.
2020.04.08.
   564 건의 뉴스가 저장 됨
page/total page = 55/ 

2020.04.04.
2020.04.07.
2020.04.08.
2020.04.02.
   1003 건의 뉴스가 저장 됨
page/total page = 100/ 959
2020.04.08.
2020.04.07.
2020.04.06.
2020.04.05.
2020.04.06.
2020.04.07.
2020.04.09.
2020.04.08.
2020.04.05.
2020.04.03.
   1013 건의 뉴스가 저장 됨


In [0]:
df1

date   press                                 title  \
0   2020.04.10.    한국일보    삼성바이오로직스, 미국 코로나19 치료 후보물질 위탁 생산한다   
1   2020.04.10.    매일경제              美 코로나치료 후보물질 삼성바이오가 생산한다   
2   2020.04.10.  파이낸셜뉴스                      삼바, 코로나 치료제 위탁생산   
3   2020.04.10.    서울경제  김광수 농협금융 회장 1년 더···"포스트 코로나 이끌 최적임자"   
4   2020.04.10.     한겨레         “반복적 비말 노출”…콜센터 직원 첫 ‘코로나 산재’   
5   2020.04.10.    경향신문             코로나19 감염 노동자 국내 첫 ‘산재’ 인정   
6   2020.04.10.    세계일보                구로 콜센터 상담원 코로나 산재 첫 인정   
7   2020.04.10.   아시아경제            코로나19의 비밀 풀었다..RNA전사체 첫 분석   
8   2020.04.10.    중앙일보       과연 노벨상 후보…김빛내리 코로나19 치료제 열쇠 찾았다   
9   2020.04.10.   헤럴드경제         김빛내리 교수, 세계 첫 ‘코로나 유전자 지도’ 완성   
10  2020.04.10.    세계일보                 코로나19 바이러스 유전자 비밀 풀었다   
11  2020.04.10.    매일경제             靑에 전화건 빌게이츠 "코로나 백신개발 협력"   
12  2020.04.10.  파이낸셜뉴스                "확진=해고"… '주홍글씨'가 된 코로나   
13  2020.04.10.    조선일보            은마·잠실주공 4억원 하락… 코로나가 집값 잡네   
14  2020.04.10.    한국경제                금융당국, 코로나 테마주에 '옐로 카드'   
15  2020.04.10.     한겨레          비상사태 선포 4주 미국, 코로나19 확산세 잡을까   
16  2020.04.10.     한겨레          ‘코로나 집콕’ 88살 할머니 첫 외출 나와 한 표   

                                             contents  \
0   “4400억원 규모… 2021년 3공장서 생산 예정”인천 송도에 있는 삼성바이오로직...   
1   美비어社와 4400억원 계약상장이래 최대규모 수주따내김태한 사장 삼성바이오로직스가 ...   
2   美 비어바이오와 4400억원 계약  전 세계적으로 코로나19 치료제 생산이 임박한 ...   
3   임추위, 만장일치로 연임 추천2년 연속 '1조대 순익' 긍정 평가金, 디지털·생존전...   
4   집단감염 구로 콜센터 밀집공간 “업무와 상당한 인과관계 있다” 발병경로 확인돼 3주...   
5   ㆍ구로 콜센터 직원 1명 신속 판정ㆍ간호사·조무사 3명도 심의 중ㆍ‘업무 연관’ 추...   
6   대구 52일 만에 신규 환자 ‘0’ 기록서울 구로구 콜센터에서 일하다 코로나19 확...   
7   사스코로나바이러스-2의 생활사[아시아경제 황준호 기자] 국내 연구진이 신종 코로나바...   
8   장혜식 교수와 연구, 세계 첫 성과코로나 RNA 전사체 최초로 분석고해상도 유전자 ...   
9   바이러스 RNA 전사체 분석진단키트 개선·치료제 개발 가속국내 과학계서 노벨상 후보...   
10  IBS·국립보건연구원 공동연구팀 / 노벨상 유력평가 김빛내리 단장 / RNA 변형 ...   
11  25분 통화…"한국이 모범" 마이크로소프트 창업자인 빌 게이츠 '빌&멀린다 게이츠재...   
12  해고·권고사직 종용 사례 증가구로 콜센터 직원들 실직 걱정확진자 접촉 회사에 숨기기...   
13  [19번 부동산 대책에도 못잡았는데…] 서울 강남 재건축 4개월 만에 급락은마 23...   
14  금융위원회는 급등락을 거듭하는 ‘코로나19 테마주’에 대한 모니터링과 불공정거래 단...   
15  [토요판] 다음주의 질문한국에서 출발하는 모든 미국행 항공기 승객에 대해 코로나19...   
16  사전투표 첫날 533만명 투표제21대 국회의원 선거 사전투표 첫날인 10일 오전 서...   

                                                 link  
0   https://news.naver.com/main/read.nhn?mode=LSD&...  
1   https://news.naver.com/main/read.nhn?mode=LSD&...  
2   https://news.naver.com/main/read.nhn?mode=LSD&...  
3   https://news.naver.com/main/read.nhn?mode=LSD&...  
4   https://news.naver.com/main/read.nhn?mode=LSD&...  
5   https://news.naver.com/main/read.nhn?mode=LSD&...  
6   https://news.naver.com/main/read.nhn?mode=LSD&...  
7   https://news.naver.com/main/read.nhn?mode=LSD&...  
8   https://news.naver.com/main/read.nhn?mode=LSD&...  
9   https://news.naver.com/main/read.nhn?mode=LSD&...  
10  https://news.naver.com/main/read.nhn?mode=LSD&...  
11  https://news.naver.com/main/read.nhn?mode=LSD&...  
12  https://news.naver.com/main/read.nhn?mode=LSD&...  
13  https://news.naver.com/main/read.nhn?mode=LSD&...  
14  https://news.naver.com/main/read.nhn?mode=LSD&...  
15  https://news.naver.com/main/read.nhn?mode=LSD&...  
16  https://news.naver.com/main/read.nhn?mode=LSD&...

In [0]:
saveFname = 'COVID_200401_200410.csv'
df1.to_csv(saveFname, encoding='utf-8')

In [0]:
query  = "코로나"
s_date = "2020.03.21"
e_date = "2020.03.31"
maxpage = 101

news2 = get_info(query, s_date, e_date, maxpage)
df2 = news2.crawler()

page/total page = 1/ 932
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
   19 건의 뉴스가 저장 됨
page/total page = 2/ 932
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.30.
2020.03.31.
2020.03.30.
2020.03.31.
2020.03.31.
2020.03.31.
   33 건의 뉴스가 저장 됨
page/total page = 3/ 932
2020.03.31.
2020.03.31.
2020.03.29.
2020.03.29.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.30.
2020.03.31.
2020.03.31.
2020.03.31.
   44 건의 뉴스가 저장 됨
page/total page = 4/ 932
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.30.
2020.03.31.
2020.03.30.
2020.03.31.
2020.03.30.
   54 건의 뉴스가 저장 됨
page/total page = 5/ 932
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.30.
2020.03.30.
2020.03.30.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.31.

2020.03.30.
   507 건의 뉴스가 저장 됨
page/total page = 49/ 932
2020.03.30.
2020.03.29.
2020.03.26.
2020.03.27.
2020.03.31.
2020.03.31.
2020.03.31.
2020.03.30.
2020.03.30.
2020.03.30.
   517 건의 뉴스가 저장 됨
page/total page = 50/ 932
2020.03.29.
2020.03.31.
2020.03.25.
2020.03.31.
2020.03.30.
2020.03.28.
2020.03.31.
2020.03.26.
2020.03.30.
2020.03.29.
   527 건의 뉴스가 저장 됨
page/total page = 51/ 932
2020.03.30.
2020.03.30.
2020.03.29.
2020.03.31.
2020.03.31.
2020.03.27.
2020.03.29.
2020.03.29.
2020.03.30.
2020.03.31.
   537 건의 뉴스가 저장 됨
page/total page = 52/ 932
2020.03.30.
2020.03.30.
2020.03.30.
2020.03.31.
2020.03.29.
2020.03.29.
2020.03.27.
2020.03.27.
2020.03.27.
2020.03.30.
2020.03.27.
2020.03.30.
   549 건의 뉴스가 저장 됨
page/total page = 53/ 932
2020.03.29.
2020.03.27.
2020.03.27.
2020.03.27.
2020.03.27.
2020.03.27.
2020.03.29.
2020.03.29.
2020.03.30.
2020.03.30.
2020.03.30.
2020.03.30.
   561 건의 뉴스가 저장 됨
page/total page = 54/ 932
2020.03.30.
2020.03.30.
2020.03.25.
2020.03.30.
2020.03.30.
2020.03.30

2020.03.27.
2020.03.26.
2020.03.29.
2020.03.25.
2020.03.26.
2020.03.27.
2020.03.25.
2020.03.25.
2020.03.23.
   1014 건의 뉴스가 저장 됨
page/total page = 98/ 932
2020.03.25.
2020.03.27.
2020.03.23.
2020.03.26.
2020.03.26.
2020.03.27.
2020.03.25.
2020.03.26.
2020.03.26.
2020.03.24.
   1024 건의 뉴스가 저장 됨
page/total page = 99/ 932
2020.03.27.
2020.03.28.
2020.03.24.
2020.03.24.
2020.03.25.
2020.03.27.
2020.03.22.
2020.03.24.
2020.03.25.
2020.03.24.
2020.03.23.
2020.03.23.
   1036 건의 뉴스가 저장 됨
page/total page = 100/ 932
2020.03.24.
2020.03.28.
2020.03.26.
2020.03.23.
2020.03.25.
2020.03.27.
2020.03.25.
2020.03.25.
2020.03.25.
2020.03.22.
   1046 건의 뉴스가 저장 됨


In [0]:
saveFname = 'COVID_200321_200331.csv'
df2.to_csv(saveFname, encoding='utf-8')

In [0]:
df2

date   press                                              title  \
0     2020.03.31.     한겨레                                 코로나19 ‘혈장 치료’ 추진한다   
1     2020.03.31.    경향신문            코로나19 완치자 혈장 뽑아 치료법 찾는다 [‘코로나19’ 확산 비상]   
2     2020.03.31.    전자신문            코로나19 장기화 시 업종별 매출 24% 급감…'기업 규제 완화' 시급   
3     2020.03.31.     한겨레                 “코로나19 유행 두달 간 주요 업종 매출액 17.5% 감소”   
4     2020.03.31.    한국일보                 전경련 “코로나 6개월 더 가면 주요산업 연매출 24% 급감”   
5     2020.03.31.    세계일보                       코로나 탓에… 업종 10개 중 9개 실적 악화 실감   
6     2020.03.31.    한국경제                      LG, 印尼 코로나 진단 돕는다…진단키트 5만개 기부   
7     2020.03.31.    문화일보                       가전제품 공급·치료센터 제공… LG ‘코로나 상생’   
8     2020.03.31.    전자신문  [코로나19 함께 넘자]LG그룹 4개 계열사, 인도네시아에 코로나19 검사키트 5만...   
9     2020.03.31.  디지털타임스                                    일자리 없애기 시작한 코로나   
10    2020.03.31.    국민일보                      코로나에 빼앗긴 참정권… 재외국민 47% 투표 못한다   
11    2020.03.31.    문화일보  <10문10답>코로나 속 21대 총선 투표 Q&A;…마스크 착용→입구서 발열 체크→...   
12    2020.03.31.    동아일보                               ‘불주사’와 코로나[횡설수설/서영아]   
13    2020.03.31.  파이낸셜뉴스               코로나 피해 중기·소상공인, 1일부터 만기연장·이자상환 유예 신청   
14    2020.03.31.   아시아경제                코로나 피해 中企·소상공 내일부터 대출만기 연장, 이자상환 유예   
15    2020.03.31.    경향신문     2월 생산 -3.5%·소비 -6%…9년 만에 최대 감소 [‘코로나19’ 확산 비상]   
16    2020.03.31.  파이낸셜뉴스  2월 생산·소비·투자 ‘트리플 약세’… 3~4월 지표는 더 나쁘다 [코로나19 경제...   
17    2020.03.31.    서울경제                      코로나19 쇼크에···생산·소비·투자 '트리플 추락'   
18    2020.03.31.    서울경제                       "코로나 의료 교류 강화" G20 통상장관 공동성명   
19    2020.03.31.  디지털타임스                          SKB·문체부 "우리함께 코로나19 이겨내요"   
20    2020.03.31.  파이낸셜뉴스                           SKB, Btv서 '코로나19 극복 캠페인'   
21    2020.03.31.    전자신문       [코로나 19 함께 넘자]SK브로드밴드, 문화체육관광부와 코로나19 극복 캠페인   
22    2020.03.31.  파이낸셜뉴스                                 코로나에 오피스 시장도 내려앉았다   
23    2020.03.31.    조선일보                      美제재 엎친데 코로나 덮쳐… 화웨이 세계4위로 급추락   
24    2020.03.31.    한국경제                      LG, 印尼 코로나 진단 돕는다…진단키트 5만개 기부   
25    2020.03.31.    문화일보                       가전제품 공급·치료센터 제공… LG ‘코로나 상생’   
26    2020.03.31.    전자신문  [코로나19 함께 넘자]LG그룹 4개 계열사, 인도네시아에 코로나19 검사키트 5만...   
27    2020.03.30.    서울경제                        서울 학원가 '코로나 비상'...강사들 잇단 확진   
28    2020.03.31.    서울신문                     코로나로 참았던 분양… 4월 5만 2079가구 쏟아진다   
29    2020.03.30.    전자신문                    코로나가 바꾼 소비…지난달 유통업 매출 절반 온라인 차지   
...           ...     ...                                                ...   
1016  2020.03.23.  디지털타임스                         DGIST, 코로나 극복 4000만원 성금 전달   
1017  2020.03.26.    문화일보                       경주 식당發 20명 확진… 코로나 ‘외식감염’ 비상   
1018  2020.03.26.    경향신문             일, 올림픽 미뤄지니 “감염 폭발 임박” [‘코로나19’ 확산 비상]   
1019  2020.03.27.   헤럴드경제                          미·중, 코로나 ‘공조 기회’ 최소 두번 날려   
1020  2020.03.25.    한국경제                             SCM생명과학, 코로나 치료제 임상 추진   
1021  2020.03.26.    한국일보                    코로나19 사태에 軍 투입한 제3세계 권위주의 강화 우려   
1022  2020.03.26.  디지털타임스                               불매 맞은 일본車, 코로나까지 앓는다   
1023  2020.03.24.    한국경제                       KT, 빅데이터로 코로나 확산 예측 모델 개발 추진   
1024  2020.03.27.   헤럴드경제                             하나금투 “코로나 증시 최후승자는 개인”   
1025  2020.03.28.    조선일보                           與 "코로나 모범국", 野 "경제실정 심판"   
1026  2020.03.24.  파이낸셜뉴스                             KT, 코로나 확산예측 공동 연구 나선다   
1027  2020.03.24.    문화일보                  코로나 확산 예측 공동 연구… KT, 서울대 등과 협업 착수   
1028  2020.03.25.    한국경제              연임 성공한 손태승 우리금융 회장, 코로나 금융지원 현장부터 찾았다   
1029  2020.03.27.    국민일보                  마스크 제작·도시락 배달… 코로나 있는 곳, 자원봉사자 있다   
1030  2020.03.22.  디지털타임스                         외투기업 41% "코로나 장기화땐 韓사업 축소"   
1031  2020.03.24.  파이낸셜뉴스                               코로나 확산에 문 닫는 부동산중개업소   
1032  2020.03.25.    경향신문      손발 묶인 ‘전자·자동차’…올해 실적 반토막 날 수도 [‘코로나19’ 확산 비상]   
1033  2020.03.24.    동아일보                 코로나에 힘 못쓴 기저효과… 3월 하루평균 수출 0.4% 감소   
1034  2020.03

In [0]:
query  = "코로나"
s_date = "2020.03.11"
e_date = "2020.03.20"
maxpage = 101

news3 = get_info(query, s_date, e_date, maxpage)
df3 = news3.crawler()

page/total page = 1/ 954
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
   12 건의 뉴스가 저장 됨
page/total page = 2/ 954
2020.03.19.
2020.03.20.
2020.03.19.
2020.03.19.
2020.03.19.
2020.03.20.
2020.03.19.
2020.03.19.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
   24 건의 뉴스가 저장 됨
page/total page = 3/ 954
2020.03.20.
2020.03.20.
2020.03.19.
2020.03.20.
2020.03.19.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
   34 건의 뉴스가 저장 됨
page/total page = 4/ 954
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.20.
2020.03.19.
2020.03.19.
2020.03.19.
2020.03.20.
2020.03.19.
2020.03.19.
2020.03.20.
   46 건의 뉴스가 저장 됨
page/total page = 5/ 954
2020.03.20.
2020.03.20.
2020.03.19.
2020.03.19.
2020.03.20.
2020.03.19.
2020.03.19.
2020.03.19.
2020.03.20.
2020.03.20.
2020.03.18.
2020.03.18.
2020.03.20.
   59 건의 뉴스가 저장 됨
page/total page = 6/ 954
2020.03.19.
2020.03.19.
2020.03.20.
2020.03.18.
2020

2020.03.19.
2020.03.19.
2020.03.19.
2020.03.19.
2020.03.19.
2020.03.18.
2020.03.19.
2020.03.16.
2020.03.18.
2020.03.18.
   505 건의 뉴스가 저장 됨
page/total page = 52/ 954
2020.03.18.
2020.03.19.
2020.03.15.
2020.03.19.
2020.03.20.
2020.03.18.
2020.03.16.
2020.03.18.
2020.03.18.
2020.03.18.
   515 건의 뉴스가 저장 됨
page/total page = 53/ 954
2020.03.18.
2020.03.19.
2020.03.19.
2020.03.19.
2020.03.18.
2020.03.18.
2020.03.17.
2020.03.18.
2020.03.17.
2020.03.17.
   525 건의 뉴스가 저장 됨
page/total page = 54/ 954
2020.03.20.
2020.03.19.
2020.03.16.
2020.03.18.
2020.03.18.
2020.03.20.
2020.03.19.
   532 건의 뉴스가 저장 됨
page/total page = 55/ 954
2020.03.18.
2020.03.16.
2020.03.17.
2020.03.18.
2020.03.19.
2020.03.19.
2020.03.20.
2020.03.18.
2020.03.18.
2020.03.16.
   542 건의 뉴스가 저장 됨
page/total page = 56/ 954
2020.03.18.
2020.03.19.
2020.03.16.
2020.03.18.
2020.03.19.
2020.03.20.
2020.03.17.
2020.03.19.
   550 건의 뉴스가 저장 됨
page/total page = 57/ 954
2020.03.19.
2020.03.18.
2020.03.20.
2020.03.18.
2020.03.19.
2020.03.19

In [0]:
saveFname = 'COVID_200311_200320.csv'
df3.to_csv(saveFname, encoding='utf-8')

In [0]:
query  = "코로나"
s_date = "2020.03.01"
e_date = "2020.03.10"
maxpage = 101

news4 = get_info(query, s_date, e_date, maxpage)
df4 = news4.crawler()

page/total page = 1/ 872
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
   17 건의 뉴스가 저장 됨
page/total page = 2/ 872
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.09.
   34 건의 뉴스가 저장 됨
page/total page = 3/ 872
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
   46 건의 뉴스가 저장 됨
page/total page = 4/ 872
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
   57 건의 뉴스가 저장 됨
page/total page = 5/ 872
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.09.
2020.03.08.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.

2020.03.09.
2020.03.10.
2020.03.08.
2020.03.10.
2020.03.10.
2020.03.09.
   512 건의 뉴스가 저장 됨
page/total page = 49/ 872
2020.03.09.
2020.03.10.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.09.
   521 건의 뉴스가 저장 됨
page/total page = 50/ 872
2020.03.09.
2020.03.10.
2020.03.10.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.08.
   531 건의 뉴스가 저장 됨
page/total page = 51/ 872
2020.03.09.
2020.03.10.
2020.03.09.
2020.03.09.
2020.03.08.
2020.03.08.
2020.03.09.
2020.03.10.
2020.03.10.
2020.03.10.
2020.03.10.
   542 건의 뉴스가 저장 됨
page/total page = 52/ 872
2020.03.10.
2020.03.09.
2020.03.05.
2020.03.05.
2020.03.10.
2020.03.09.
2020.03.09.
2020.03.09.
2020.03.08.
2020.03.10.
2020.03.09.
   553 건의 뉴스가 저장 됨
page/total page = 53/ 872
2020.03.05.
2020.03.09.
2020.03.08.
2020.03.08.
2020.03.09.
2020.03.09.
2020.03.10.
2020.03.10.
2020.03.08.
   562 건의 뉴스가 저장 됨
page/total page = 54/ 872
2020.03.08.
2020.03.09.
2020.03.09.
2020.03.08.
2020.03.05

2020.03.09.
2020.03.08.
2020.03.08.
2020.03.03.
2020.03.03.
   1010 건의 뉴스가 저장 됨
page/total page = 98/ 872
2020.03.09.
2020.03.05.
2020.03.04.
2020.03.06.
2020.03.05.
2020.03.04.
2020.03.06.
2020.03.05.
2020.03.06.
2020.03.04.
   1020 건의 뉴스가 저장 됨
page/total page = 99/ 872
2020.03.04.
2020.03.05.
2020.03.05.
2020.03.07.
2020.03.06.
2020.03.05.
2020.03.08.
2020.03.07.
2020.03.10.
2020.03.04.
   1030 건의 뉴스가 저장 됨
page/total page = 100/ 872
2020.03.03.
2020.03.08.
2020.03.08.
2020.03.01.
2020.03.09.
2020.03.09.
2020.03.08.
2020.03.02.
2020.03.05.
2020.03.05.
   1040 건의 뉴스가 저장 됨


In [0]:
saveFname = 'COVID_200301_200310.csv'
df4.to_csv(saveFname, encoding='utf-8')